In [ ]:

from google.colab import drive
drive.mount('/content/drive')

# Imports
import os
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    confusion_matrix, roc_auc_score, roc_curve
)
import numpy as np
from collections import Counter

# Count and print number of images per class in a dataset split
def count_labels(dataset, class_names, split_name=""):
    labels = [sample[1] for sample in dataset]  # extract label index
    label_counts = Counter(labels)
    print(f"📊 {split_name} split distribution:")
    for idx in range(len(class_names)):
        print(f"   📁 {class_names[idx]}: {label_counts[idx]} images")
    print()

# Load dataset and print total images
def load_food_dataset(base_path="/content/drive/MyDrive/Dataset/Dataset/Combined/Our dataset", img_size=224, batch_size=32, split_ratio=0.8):
    transform = transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
        transforms.Normalize([0.5]*3, [0.5]*3)
    ])

    full_dataset = datasets.ImageFolder(root=base_path, transform=transform)
    class_names = full_dataset.classes
    total_images = len(full_dataset)
    print(f"📦 Total images found: {total_images} across {len(class_names)} classes: {class_names}")

    train_size = int(split_ratio * total_images)
    val_size = total_images - train_size
    train_set, val_set = random_split(full_dataset, [train_size, val_size])

    print(f"🟢 Training images: {len(train_set)}")
    count_labels(train_set, class_names, split_name="Training")

    print(f"🔵 Validation images: {len(val_set)}")
    count_labels(val_set, class_names, split_name="Validation")

    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, class_names

# Simple CNN
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(128 * 28 * 28, 256), nn.ReLU(),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        return self.net(x)

# Basic evaluation (for val accuracy only)
def evaluate_model(model, val_loader, device):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    return 100. * correct / total

# Detailed validation metrics
def evaluate_model_detailed(model, val_loader, device):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='weighted', zero_division=0)
    recall = recall_score(all_labels, all_preds, average='weighted', zero_division=0)
    cm = confusion_matrix(all_labels, all_preds)

    try:
        probs = []
        with torch.no_grad():
            for images, _ in val_loader:
                images = images.to(device)
                outputs = model(images)
                probs.append(torch.softmax(outputs, dim=1).cpu().numpy())
        probs = np.vstack(probs)
        from sklearn.preprocessing import label_binarize
        labels_one_hot = label_binarize(all_labels, classes=list(range(probs.shape[1])))
        auc = roc_auc_score(labels_one_hot, probs, average='weighted', multi_class='ovr')
        roc_data_available = True
    except Exception as e:
        print(f"⚠️ Could not compute ROC AUC: {e}")
        auc = None
        roc_data_available = False

    return accuracy, precision, recall, cm, auc, roc_data_available, all_labels, all_preds, probs if roc_data_available else None

# Training function
def train_model(model, train_loader, val_loader, device, epochs=20, lr=0.001, save_path='model.pth'):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    train_acc_list = []
    val_acc_list = []

    start_time = time.time()
    for epoch in range(epochs):
        epoch_start = time.time()
        model.train()
        correct, total, train_loss = 0, 0, 0.0

        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} - Training"):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

        train_acc = 100. * correct / total
        val_acc = evaluate_model(model, val_loader, device)

        train_acc_list.append(train_acc)
        val_acc_list.append(val_acc)

        epoch_end = time.time()
        print(f"Epoch {epoch+1}: Train Loss = {train_loss/total:.4f}, Train Acc = {train_acc:.2f}%, Val Acc = {val_acc:.2f}%, Time = {epoch_end - epoch_start:.2f} sec")

    total_time = time.time() - start_time
    print(f"⏱ Total training time: {total_time:.2f} seconds")

    torch.save(model.state_dict(), save_path)
    print(f"✅ Model saved at: {save_path}")

    plt.figure(figsize=(8,5))
    plt.plot(train_acc_list, label='Train Accuracy')
    plt.plot(val_acc_list, label='Validation Accuracy')
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy (%)")
    plt.title("Training and Validation Accuracy")
    plt.legend()
    plt.grid(True)
    plt.show()

# Predict one image
def predict_image(model, image_path, class_names, device):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.5]*3, [0.5]*3)
    ])

    image = Image.open(image_path).convert('RGB')
    image_tensor = transform(image).unsqueeze(0).to(device)

    model.eval()
    with torch.no_grad():
        output = model(image_tensor)
        _, predicted = output.max(1)
        predicted_class = class_names[predicted.item()]

    print(f"🔎 Prediction for '{os.path.basename(image_path)}': {predicted_class}")

# Main pipeline
def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"⚙️ Using device: {device}")

    data_path = '/content/drive/MyDrive/Dataset/Dataset/Combined/Our dataset'
    model_save_path = '/content/drive/MyDrive/Dataset/Dataset/Samifood_model_basic_our_data.pth'

    start_load = time.time()
    train_loader, val_loader, class_names = load_food_dataset(base_path=data_path)
    end_load = time.time()
    print(f"⏱ Data loading time: {end_load - start_load:.2f} seconds")

    model = SimpleCNN(num_classes=len(class_names)).to(device)

    train_model(model, train_loader, val_loader, device, epochs=150, lr=1e-5, save_path=model_save_path)

    model.load_state_dict(torch.load(model_save_path, map_location=device))

    accuracy, precision, recall, cm, auc, roc_data_available, all_labels, all_preds, probs = evaluate_model_detailed(model, val_loader, device)

    print(f"\n📊 Validation Metrics:\nAccuracy: {accuracy:.4f}\nPrecision: {precision:.4f}\nRecall: {recall:.4f}")
    print("Confusion Matrix:")
    print(cm)

    if auc is not None:
        print(f"ROC AUC: {auc:.4f}")
        plt.figure(figsize=(8,6))
        for i in range(len(class_names)):
            fpr, tpr, _ = roc_curve(np.array(all_labels) == i, probs[:, i])
            plt.plot(fpr, tpr, label=f"Class {class_names[i]} (AUC = {roc_auc_score(np.array(all_labels) == i, probs[:, i]):.2f})")
        plt.plot([0,1],[0,1],'k--')
        plt.xlabel("False Positive Rate")
        plt.ylabel("True Positive Rate")
        plt.title("ROC Curve - Multiclass")
        plt.legend(loc='lower right')
        plt.grid(True)
        plt.show()
    else:
        print("ROC AUC curve not available.")

    test_image_path = "/content/drive/MyDrive/Dataset/Dataset/Combined/Our dataset/Sugarcane Juice/f52981ca1c92d937fd3fd6fc9eb26215.jpg"  # Change path as needed
    start_pred = time.time()
    predict_image(model, test_image_path, class_names, device)
    end_pred = time.time()
    print(f"⏱ Prediction time: {end_pred - start_pred:.4f} seconds")

if __name__ == "__main__":
    main()

Mounted at /content/drive
⚙️ Using device: cpu
📦 Total images found: 1475 across 10 classes: ['Fuchka', 'Jhalmuri', 'Kalavuna', 'Khichuri', 'Mishti', 'Pitha', 'Puri', 'Roshmalai', 'Shingara', 'Sugarcane Juice']
🟢 Training images: 1180
📊 Training split distribution:
   📁 Fuchka: 117 images
   📁 Jhalmuri: 120 images
   📁 Kalavuna: 123 images
   📁 Khichuri: 126 images
   📁 Mishti: 119 images
   📁 Pitha: 127 images
   📁 Puri: 121 images
   📁 Roshmalai: 78 images
   📁 Shingara: 124 images
   📁 Sugarcane Juice: 125 images

🔵 Validation images: 295
📊 Validation split distribution:
   📁 Fuchka: 35 images
   📁 Jhalmuri: 32 images
   📁 Kalavuna: 29 images
   📁 Khichuri: 26 images
   📁 Mishti: 33 images
   📁 Pitha: 25 images
   📁 Puri: 31 images
   📁 Roshmalai: 29 images
   📁 Shingara: 28 images
   📁 Sugarcane Juice: 27 images

⏱ Data loading time: 891.74 seconds


Epoch 1/150 - Training: 100%|██████████| 37/37 [03:23<00:00,  5.49s/it]


Epoch 1: Train Loss = 2.2350, Train Acc = 29.32%, Val Acc = 37.29%, Time = 228.24 sec


Epoch 2/150 - Training: 100%|██████████| 37/37 [03:24<00:00,  5.53s/it]


Epoch 2: Train Loss = 2.0187, Train Acc = 51.61%, Val Acc = 49.15%, Time = 230.68 sec


...

Epoch 139/150 - Training: 100%|██████████| 37/37 [03:29<00:00,  5.66s/it]


Epoch 139: Train Loss = 0.0141, Train Acc = 99.66%, Val Acc = 62.71%, Time = 235.61 sec


Epoch 140/150 - Training: 100%|██████████| 37/37 [03:28<00:00,  5.63s/it]


Epoch 140: Train Loss = 0.0190, Train Acc = 99.58%, Val Acc = 64.07%, Time = 235.60 sec


Epoch 141/150 - Training:  43%|████▎     | 16/37 [01:28<01:54,  5.45s/it]